In [1]:
import cv2
import os

cam = cv2.VideoCapture(0)


face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# For each person, enter one numeric face id
face_id = input('\n Make sure the first user entered is 0.enter user id end press <return> ==>  ')

print("\n [INFO] Initializing face capture. Look the camera and wait ...")
# Initialize individual sampling face count
count = 0

while(True):

    ret, img = cam.read()
    img = cv2.flip(img, 1) # flip video image vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w+50,y+h+50), (255,0,0), 2)     
        count += 1

        # Save the captured image into the datasets folder
        gray = gray[y:y+h,x:x+w]
        img_path ="dataset/User." + str(face_id) + '.' + str(count) + ".jpg"
        cv2.imwrite(img_path,gray )

        cv2.imshow('image', img)

    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 70: # Take 70 face sample and stop video
         break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()





 Make sure the first user entered is 0.enter user id end press <return> ==>  0

 [INFO] Initializing face capture. Look the camera and wait ...

 [INFO] Exiting Program and cleanup stuff


In [2]:
import cv2
import numpy as np
from PIL import Image
import os
import numpy as np
import cv2
import os
import h5py
import dlib
from imutils import face_utils
from keras.models import load_model
import sys
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dropout
from keras.layers import Dense, Activation, Flatten
from keras.utils import to_categorical
from keras import backend as K 
from sklearn.model_selection import train_test_split
from Model import model
from keras import callbacks
#import tensorflow as tf

# Path for face image database
path = 'dataset'

recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");


def downsample_image(img):
    img = Image.fromarray(img.astype('uint8'), 'L')
    img = img.resize((32,32), Image.ANTIALIAS)
    return np.array(img)



# function to get the images and label data
def getImagesAndLabels(path):
    
    path = 'dataset'
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []

    for imagePath in imagePaths:
        
        #if there is an error saving any jpegs
        try:
            PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        except:
            continue    
        img_numpy = np.array(PIL_img,'uint8')

        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faceSamples.append(img_numpy)
        ids.append(id)
    return faceSamples,ids

print ("\n [INFO] Training faces now.")
faces,ids = getImagesAndLabels(path)

K.clear_session()
n_faces = len(set(ids))
model = model((32,32,1),n_faces)
faces = np.asarray(faces)
faces = np.array([downsample_image(ab) for ab in faces])
ids = np.asarray(ids)
faces = faces[:,:,:,np.newaxis]
print("Shape of Data: " + str(faces.shape))
print("Number of unique faces : " + str(n_faces))


ids = to_categorical(ids)

faces = faces.astype('float32')
faces /= 255.

x_train, x_test, y_train, y_test = train_test_split(faces,ids, test_size = 0.2, random_state = 0)

checkpoint = callbacks.ModelCheckpoint('model.h5',monitor='val_acc',
                                           save_best_only=True, save_weights_only=True, verbose=1)
                                    
model.fit(x_train, y_train,
             batch_size=32,
             epochs=15,
             validation_data=(x_test, y_test),
             shuffle=True,callbacks=[checkpoint])
model.save_weights('my_model_weights.h5')

             

# Print the numer of faces trained and end program
print("\n [INFO] " + str(n_faces) + " faces trained. Exiting Program")



 [INFO] Training faces now.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        4160      
_________________________________________________________________
dropout (Dropout)          

C:\Users\prince\anaconda3\envs\dlib\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Epoch 1/15
6/6 [==============================] - 4s 704ms/step - loss: 4.2623 - accuracy: 0.3818 - val_loss: 1.0959 - val_accuracy: 0.3810
Epoch 2/15
6/6 [==============================] - 4s 601ms/step - loss: 0.6721 - accuracy: 0.6788 - val_loss: 1.1001 - val_accuracy: 0.3571
Epoch 3/15
6/6 [==============================] - 3s 577ms/step - loss: 0.5477 - accuracy: 0.7152 - val_loss: 1.0950 - val_accuracy: 0.3095
Epoch 4/15
6/6 [==============================] - 3s 579ms/step - loss: 0.4396 - accuracy: 0.7576 - val_loss: 1.0908 - val_accuracy: 0.6667
Epoch 5/15
6/6 [==============================] - 4s 598ms/step - loss: 0.3167 - accuracy: 0.8788 - val_loss: 1.0845 - val_accuracy: 0.6667
Epoch 6/15
6/6 [==============================] - 3s 581ms/step - loss: 0.4063 - accuracy: 0.7879 - val_loss: 1.0824 - val_accuracy: 0.6667
Epoch 7/15
6/6 [==============================] - 4s 589ms/step - loss: 0.2742 - accuracy: 0.9212 - val_loss: 1.0797 - val_accuracy: 0.4286
Epoch 8/15
6/6 [====

In [ ]:
import numpy as np
import cv2
import os
import h5py
import dlib
from imutils import face_utils
from keras.models import load_model
import sys
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dropout
from keras.layers import Dense, Activation, Flatten
from PIL import Image
from Model import model



def getImagesAndLabels():
    
    path = 'dataset'
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []

    for imagePath in imagePaths:

        #if there is an error saving any jpegs
        try:
            PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        except:
            continue  
        img_numpy = np.array(PIL_img,'uint8')

        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faceSamples.append(img_numpy)
        ids.append(id)
    return faceSamples,ids
    

_,ids = getImagesAndLabels()
model = model((32,32,1),len(set(ids)))
model.load_weights  ('my_model_weights.h5')
model.summary()

cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
font = cv2.FONT_HERSHEY_SIMPLEX
def start():
    cap = cv2.VideoCapture(0)
    print('here')
    ret = True

    clip = []
    while ret:
        #read frame by frame
        ret, frame = cap.read()
        nframe = frame
        faces = faceCascade.detectMultiScale(
        frame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30))

        try:
            (x,y,w,h) = faces[0]
        except:
            continue
        frame = frame[y:y+h,x:x+w]
        frame = cv2.resize(frame, (32,32))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('result small' , frame)
        c= cv2.waitKey(1)
        if c & 0xFF == ord('q'):
            break
        
        #gray = gray[np.newaxis,:,:,np.newaxis]
        gray = gray.reshape(-1, 32, 32, 1).astype('float32') / 255.
        print(gray.shape)
        prediction = model.predict(gray)
        print("prediction:" + str(prediction))

        
        print("\n\n\n\n")
        print("----------------------------------------------")
        #labels = ['George W Bush' ,'Rishabh']
        prediction = prediction.tolist()
        
        listv = prediction[0]
        n = listv.index(max(listv))
        print("\n")
        print("----------------------------------------------")
        #print( "Highest Probability: " + labels[n] + "==>" + str(prediction[0][n]) )
        print( "Highest Probability: " + "User " + str(n) + "==>" + str(prediction[0][n]) )
        
        print("----------------------------------------------")
        print("\n")
        for (x, y, w, h) in faces:
            try:
                cv2.rectangle(nframe, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(nframe, str(labels[n]), (x+5,y-5), font, 1, (255,255,255), 2)
                #cv2.putText(nframe, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)
            except:
                la = 2 
        prediction = np.argmax(model.predict(gray), 1)
        print(prediction)
        cv2.imshow('result', nframe)
        c = cv2.waitKey(1)
        if c & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    
    
start()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
activation_6 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
activation_7 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 64)        4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 28, 28, 64)       

(1, 32, 32, 1)
prediction:[[0.32699096 0.32643592 0.34657317]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.3465731739997864
----------------------------------------------


[2]
(1, 32, 32, 1)
prediction:[[0.32707316 0.32610506 0.34682178]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.34682178497314453
----------------------------------------------


[2]
(1, 32, 32, 1)
prediction:[[0.3283069  0.3270125  0.34468058]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.3446805775165558
----------------------------------------------


[2]
(1, 32, 32, 1)
prediction:[[0.32788768 0.32649016 0.34562215]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.34562

(1, 32, 32, 1)
prediction:[[0.32686454 0.32707763 0.34605777]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.3460577726364136
----------------------------------------------


[2]
(1, 32, 32, 1)
prediction:[[0.32750568 0.32617813 0.3463162 ]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.3463161885738373
----------------------------------------------


[2]
(1, 32, 32, 1)
prediction:[[0.3261024  0.32681006 0.3470875 ]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.3470875024795532
----------------------------------------------


[2]
(1, 32, 32, 1)
prediction:[[0.3253089 0.3261501 0.348541 ]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.348540991

(1, 32, 32, 1)
prediction:[[0.32944706 0.32675076 0.34380218]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.34380218386650085
----------------------------------------------


[2]
(1, 32, 32, 1)
prediction:[[0.32991806 0.32692328 0.34315872]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.3431587219238281
----------------------------------------------


[2]
(1, 32, 32, 1)
prediction:[[0.32916477 0.32647842 0.34435683]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.34435683488845825
----------------------------------------------


[2]
(1, 32, 32, 1)
prediction:[[0.32828262 0.3269652  0.34475222]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 2==>0.3447